Test model

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from model import Model
path = '../data/train_data.pkl'
path2 = '../data/val_data.pkl'
noisy_imgs_1, noisy_imgs_2 = torch.load(path)
noisy_imgs_1, noisy_imgs_2 = noisy_imgs_1.float(), noisy_imgs_2.float()
test, truth = torch.load(path)
test, truth = test.float(), truth.float()

In [3]:
def sample(tensor1, tensor2, k):
    perm = torch.randperm(tensor1.size(0))
    idx = perm[:k]
    return tensor1[idx], tensor2[idx]

In [6]:
m = Model()
s1, s2 = sample(noisy_imgs_1, noisy_imgs_2, 500)
m.train(s1, s2, 10)

0
1
2
3
4
5
6
7
8
9


In [7]:
from others.pnsr import psnr
t1, t2 = sample(test, truth, 500)

In [8]:
psnr(m.predict(t1)/256, t2/256)

tensor(5.8344, grad_fn=<MulBackward0>)